# NLP with Logistic Regression

**Objective:** Given a tweet, predict if it has **Positive** sentiment or **Negative** sentiment.

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

from nltk.corpus import twitter_samples, stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import re
import string

In [2]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [3]:
# Split data into train, test set (80-20%)
train_pos = all_positive_tweets[:4000]
train_neg = all_negative_tweets[:4000]

test_pos = all_positive_tweets[4000:]
test_neg = all_negative_tweets[4000:]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [4]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [5]:
print(f"train_y.shape = {train_y.shape}")
print(f"test_y.shape = {test_y.shape}")

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


#### Necessary functions

In [6]:
def build_freqs(tweets, labels):
    freqs = {}
    for tweet, label in zip(tweets, np.squeeze(labels)):
        for word in process_tweet(tweet):
            pair = (word, label)
            freqs[pair] = freqs.get(pair, 0) + 1
    return freqs

In [7]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_eng = stopwords.words('english')
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*','', tweet)
    tweet = re.sub(r'#', '', tweet)
    
    tweet_tokens = tokenizer.tokenize(tweet)
    tweet_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_eng) and (word not in string.punctuation):
            stem_word = stemmer.stem(word)
            tweet_clean.append(stem_word)
    return tweet_clean    

In [9]:
freqs = build_freqs(train_x, train_y)

print(f"type(freqs)= {type(freqs)}")
print(f"len(freqs)= {len(freqs)}")

type(freqs)= <class 'dict'>
len(freqs)= 11346


In [10]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


### Logistic Regression

In [11]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [12]:
def gradientDescent(x, y, theta, alpha, num_iters):
    m = x.shape[0]
    
    for i in range(0, num_iters):
        z = np.dot(x, theta)
        
        h = sigmoid(z)
        
        J = (-1/m) * (np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1 - h)))
        
        theta = theta - alpha/m * (np.dot(x.T, (h - y)))
    
    J = float(J)
    return J, theta

In [13]:
def extract_features(tweet, freqs):
    clean_tweet = process_tweet(tweet)
    
    x = np.zeros((1, 3))
    
    x[0, 0] = 1
    
    for word in clean_tweet:
        x[0, 1] += freqs.get((word, 1.0), 0)
        
        x[0, 2] += freqs.get((word, 0.0), 0)
    
    return x

In [14]:
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


In [15]:
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


In [21]:
# Train your model
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x[i], freqs)

Y = train_y

J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.24216529.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]


In [22]:
def predict_tweet(tweet, freqs, theta):
    x = extract_features(tweet, freqs)
    
    y_pred = sigmoid(np.dot(x, theta))
    
    return y_pred

In [23]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

I am happy -> 0.518580
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530898
great great great -> 0.546273
great great great great -> 0.561561


In [24]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)

    
    return accuracy

In [25]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950
